<a href="https://colab.research.google.com/github/nntadotzip/vi-augmentation/blob/main/BERTbase-cased_SentencePairClassification_AugmentedDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Install

In [1]:
!pip install datasets==1.0.1
!pip install transformers==3.1.0

     |████████████████████████████████| 1.8 MB 4.3 MB/s 
     |████████████████████████████████| 243 kB 52.1 MB/s 
     |████████████████████████████████| 884 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 3.0 MB 22.5 MB/s 
     |████████████████████████████████| 1.2 MB 44.2 MB/s 


In [2]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric

os.environ["TOKENIZERS_PARALLELISM"] = "false"

PyTorch version 1.10.0+cu111 available.
TensorFlow version 2.7.0 available.


In [3]:
# Check that we are using 100% of GPU memory footprint support libraries/code
# from https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil
!pip -q install psutil
!pip -q install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.9 GB  | Proc size: 1.6 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


# Dataset

In [4]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('pair-backtranslations_init277_to518_vi-en-fr_withCosineUsingSBert.csv', encoding='utf-8')
# df['idx'] = df.index
df

,question_1,question_2,similarity,sem_sim,q1_en_to_vi,q2_fr_to_vi,cosine_usingSBert
0,"""WTO là tên viết tắt của tổ chức nào sau đây?\n|Tổ chức Thương mại thế giới.|Liên minh châu Âu.|Hiệp ước tự do thương mại Bắc Mĩ.|Diễn đàn hợp tác kinh tế châu Á – Thái Bình Dương.Đáp án""","""WTO là tên viết tắt của tổ chức|Thương mại thế giới.|Quỹ tiền tệ quốc tế.|Khu vực mậu dịch tự do châu Á.|Diễn đàn hợp tác các kinh tế châu Á -Thái Bình Dương.""",0.602151,1.0,WTO là tên sau của tổ chức sau?\n|.Tổ chức Thương mại Thế giới.|.Liên minh châu Âu.|.Hiệp ước miễn phí thương mại Bắc Mỹ.|.Diễn đàn hợp tác kinh tế châu Á-Thái Bình Dương.,WTO là một tên viết tắt của tổ chức | Thương mại thế giới. | Quỹ tiền tệ quốc tế. | Khu vực thương mại tự do châu Á. | Diễn đàn hợp tác kinh tế châu Á-Thái Bình Dương.,0.917165
1,"""Nhằm thúc đẩy sự phát triển kinh tế - xã hội các nước đang phát triển đã tiến hành\n|Nhận chuyển giao các công nghệ lỗi thời, gây ô nhiễm.|Đón đầu được công nghệ hiện đại và áp dụng vào sản xuất.|Mở rộng thị trường buôn bán với nhiều nước trên thế giới.|Phá bỏ hàng rào thuế quan, đẩy mạnh xuất khẩu hàng hóa.Đáp án""","""Trong bối cảnh toàn cầu hóa, các nước đang phát triển đã nhanh chóng nắm bắt được cơ hội nào dưới đây nhằm thúc đẩy sự phát triển kinh tế - xã hội?\n|Nhận chuyển giao các công nghệ lỗi thời, gây ô nhiễm.|Đón đầu được công nghệ hiện đại và áp dụng vào sản xuất.|Mở rộng thị trường buôn bán với nhiều nước trên thế giới.|Phá bỏ hàng rào thuế quan, đẩy mạnh xuất khẩu hàng hóa.Đáp án""",0.745407,1.0,"Để thúc đẩy phát triển kinh tế xã hội của các nước đang phát triển tiến hành\n|.Nhận chuyển giao công nghệ lỗi thời, gây ô nhiễm.|.Chào đón công nghệ hiện đại và áp dụng cho sản xuất.|.Mở rộng giao dịch thị trường với nhiều quốc gia trên thế giới.|.Tiêu diệt hàng rào thuế quan, thúc đẩy xuất khẩu hàng hóa.","Trong bối cảnh toàn cầu hóa, các nước đang phát triển đã nhanh chóng hiểu được một trong những cơ hội sau đây để thúc đẩy phát triển kinh tế xã hội?\n|.Nhận được một chuyển giao công nghệ lỗi thời, gây ô nhiễm.|.Chào mừng công nghệ sản xuất hiện đại và ứng dụng.|.Mở rộng thị trường thương mại với nhiều quốc gia trên thế giới.|.Tiêu diệt hàng rào thuế quan, thúc đẩy xuất khẩu hàng hóa.",0.958279
2,"""Đặc điểm nào không đúng với đặc điểm dân cư Hoa Kì?\n|Dân nhập cư chủ yếu có nguồn gốc Châu Phi.|Dân số Hoa Kì đông.|Dân số Hoa Kì tăng nhanh.|Dân số tăng chủ yếu do nhập cư.Đáp án""","""Đặc điểm nào sau đây không đúng với đặc điểm dân cư Hoa Kì?\n|Dân số Hoa Kì tăng nhanh.|Dân nhập cư chủ yếu có nguồn gốc Châu Phi.|Dân số Hoa Kì đông.|Dân số tăng chủ yếu do nhập cư.Đáp án""",0.682540,1.0,Những đặc điểm nào không đúng với đặc điểm dân số Hoa Kỳ?\n|.Người di cư chủ yếu ở Châu Phi.|.Hoa Kỳ Dân số Đông.|.Dân số Hoa Kỳ tăng nhanh.|.Dân số tăng chủ yếu do nhập cư.,Những đặc điểm nào sau đây không đúng với các đặc điểm của dân số Mỹ?\n|.Dân số Mỹ đang tăng nhanh.|.Người di cư chủ yếu ở Châu Phi.|.Dân số Đông Mỹ.|.Dân số đã tăng chủ yếu do nhập cư.,0.946972
3,"""Yếu tố tự nhiên nào vừa là khó khăn, nhưng đồng thời cũng là thuận lợi cho sự phát triển kinh tế của Hoa Kì?\n|Lãnh thổ rộng lớn.|Có nhiều động đất và núi lửa.|Có tài nguyên thiên nhiên đa dạng.|Có nhiều người nhập cư.Đáp án""","""Yếu tố tự nhiên nào vừa là khó khăn nhưng đồng thời cũng là thuận lợi cho sự phát triển kinh tế của Hoa Kì?\n|Có tài nguyên thiên nhiên đa dạng.|Lãnh thổ rộng lớn.|Có nhiều động đất và núi lửa.|Có nhiều người nhập cư.Đáp án""",0.715556,1.0,"Yếu tố tự nhiên nào cả khó khăn, nhưng cũng thuận tiện cho sự phát triển kinh tế của Hoa Kỳ?\n|.Lãnh thổ rộng lớn.|.Có nhiều trận động đất và núi lửa.|.Có tài nguyên thiên nhiên đa dạng.|.Có nhiều người nhập cư.","Những yếu tố tự nhiên là khó khăn, nhưng cũng thiết thực cho sự phát triển kinh tế của Hoa Kỳ?\n|.Có nhiều tài nguyên thiên nhiên.|.Lãnh thổ rộng lớn.|.Có nhiều trận động đất và núi lửa.|.Có nhiều người nhập cư.",0.973532
4,"""Một trong những thuận lợi của EU khi hình thành thị trường chung châu Âu là\n|

In [5]:
augmented_data_only = df
augmented_data_only.drop('question_1', axis='columns', inplace=True)
augmented_data_only.drop('question_2', axis='columns', inplace=True)
augmented_data_only.drop('similarity', axis='columns', inplace=True)
augmented_data_only=augmented_data_only.rename(columns={'q1_en_to_vi': 'sentence1', 'q2_fr_to_vi': 'sentence2', 'sem_sim': 'label', 'cosine_usingSBert': 'similarity'})
augmented_data_only.label = augmented_data_only.apply(lambda x: int(x.label), axis=1)
augmented_data_only

,label,sentence1,sentence2,similarity
0,1,WTO là tên sau của tổ chức sau?\n|.Tổ chức Thương mại Thế giới.|.Liên minh châu Âu.|.Hiệp ước miễn phí thương mại Bắc Mỹ.|.Diễn đàn hợp tác kinh tế châu Á-Thái Bình Dương.,WTO là một tên viết tắt của tổ chức | Thương mại thế giới. | Quỹ tiền tệ quốc tế. | Khu vực thương mại tự do châu Á. | Diễn đàn hợp tác kinh tế châu Á-Thái Bình Dương.,0.917165
1,1,"Để thúc đẩy phát triển kinh tế xã hội của các nước đang phát triển tiến hành\n|.Nhận chuyển giao công nghệ lỗi thời, gây ô nhiễm.|.Chào đón công nghệ hiện đại và áp dụng cho sản xuất.|.Mở rộng giao dịch thị trường với nhiều quốc gia trên thế giới.|.Tiêu diệt hàng rào thuế quan, thúc đẩy xuất khẩu hàng hóa.","Trong bối cảnh toàn cầu hóa, các nước đang phát triển đã nhanh chóng hiểu được một trong những cơ hội sau đây để thúc đẩy phát triển kinh tế xã hội?\n|.Nhận được một chuyển giao công nghệ lỗi thời, gây ô nhiễm.|.Chào mừng công nghệ sản xuất hiện đại và ứng dụng.|.Mở rộng thị trường thương mại với nhiều quốc gia trên thế giới.|.Tiêu diệt hàng rào thuế quan, thúc đẩy xuất khẩu hàng hóa.",0.958279
2,1,Những đặc điểm nào không đúng với đặc điểm dân số Hoa Kỳ?\n|.Người di cư chủ yếu ở Châu Phi.|.Hoa Kỳ Dân số Đông.|.Dân số Hoa Kỳ tăng nhanh.|.Dân số tăng chủ yếu do nhập cư.,Những đặc điểm nào sau đây không đúng với các đặc điểm của dân số Mỹ?\n|.Dân số Mỹ đang tăng nhanh.|.Người di cư chủ yếu ở Châu Phi.|.Dân số Đông Mỹ.|.Dân số đã tăng chủ yếu do nhập cư.,0.946972
3,1,"Yếu tố tự nhiên nào cả khó khăn, nhưng cũng thuận tiện cho sự phát triển kinh tế của Hoa Kỳ?\n|.Lãnh thổ rộng lớn.|.Có nhiều trận động đất và núi lửa.|.Có tài nguyên thiên nhiên đa dạng.|.Có nhiều người nhập cư.","Những yếu tố tự nhiên là khó khăn, nhưng cũng thiết thực cho sự phát triển kinh tế của Hoa Kỳ?\n|.Có nhiều tài nguyên thiên nhiên.|.Lãnh thổ rộng lớn.|.Có nhiều trận động đất và núi lửa.|.Có nhiều người nhập cư.",0.973532
4,1,"Một trong những lợi thế của EU khi hình thành thị trường chung châu Âu là\n|.Tăng thuế của các nước thành viên khi lưu hành hàng hóa và dịch vụ.|.Tăng cường tự do hóa người dân, hàng hóa, dịch vụ và vốn.|.Tăng cường vai trò của mỗi quốc gia khi giao dịch với các quốc gia bổ sung cho khối.|.Tăng thuế giá trị gia tăng khi lưu thông hàng hóa, tiền tệ và dịch vụ.","Một trong những lợi ích của EU trong quá trình hình thành thị trường chung châu Âu là\n|.Tăng thuế giá trị gia tăng trong quá trình lưu thông hàng hóa, tiền tệ và dịch vụ.|.Tăng cường tự do hóa các cá nhân, hàng hóa, dịch vụ và tiền bạc.|.Tăng thuế của các quốc gia thành viên trong quá trình chuyển động của hàng hóa, dịch vụ.|.Tăng cường vai trò của mỗi quốc gia trong các cuộc đàm phán với nước ngoài.",0.929395
...,...,...,...,...
549,1,"Những ý kiến nào sau đây là đúng với hoạt động thương mại nước ngoài của LB Nga?\n|.Giá trị xuất khẩu luôn luôn cân bằng với giá trị nhập khẩu.|.Hàng hóa xuất khẩu chính là hải sản, hàng hóa công nghiệp nhẹ.|.Hàng hóa nhập khẩu là dầu khí.|.Tổng kim ngạch ngoại thương tiếp tục tăng. Trả lời","Phát biểu nào sau đây là đúng đối với thương mại nước ngoài của Nga LB? | Giá trị xuất khẩu luôn được cân bằng với giá trị nhập khẩu. | Các sản phẩm xuất khẩu chính là hải sản, hàng hóa công nghiệp nhẹ. Hàng hóa nhập khẩu là mỏ dầu khí. | Tổng doanh thu từ ngoại thương. Tăng.",0.921619
550,1,"Giới hạn quan trọng nhất về điều kiện tự nhiên của Nhật Bản và tài nguyên thiên nhiên là\n|.Bờ biển dài, nhiều vũng nước.|.Khí hậu giao rõ từ Bắc đến Nam.|.Khoáng chất kém.|.Nhiều hòn đảo lớn và nhỏ nhưng xa nhau.","Trò đùa lớn nhất về điều kiện tự nhiên của Nhật Bản và tài nguyên thiên nhiên là một bờ biển dài, nhiều vũng nước vũ trụ. | Khí hậu được giao rõ ràng từ Bắc vào Nam. | Khoáng chất kém. | Nhiều hòn đảo lớn, nhỏ nhưng xa, nhưng ở xa.",0.966490
551,1,"Mùa đông kéo dài, lạnh và có rất nhiều tuyết là đặc điểm khí hậu của vùng khí hậu Nhật Bản sau đây?\n|.Bắc Nhật Bản.|.Nam Nhật Bản.|.Khu vực trung tâm Nhật Bản.|.Tiếng Nhật ven biển.","Các phần mùa đông, cái lạnh và có

**Loading labeled dataset**

In [6]:
df300 = pd.read_csv('Similarity-to-confirm_300.csv', encoding='utf-8')
df800 = pd.read_csv('Similarity-to-confirm-800.csv', encoding='utf-8')
df300.drop('Unnamed: 4',
  axis='columns', inplace=True)
df800.drop('manual test', axis='columns', inplace=True)
df800.drop('Note', axis='columns', inplace=True)
df800 = df800.rename(columns={'Yes/No': 'sem_sim'})
def yes_to_1(yes_no_na):
  if (yes_no_na == 'Y'):
    return 1
  elif (yes_no_na == 'N'):
    return 0
  else:
    return ''
df800.sem_sim = df800['sem_sim'].map(yes_to_1)

df_ = df300.append(df800, sort=False, ignore_index=True)
df_ = df_.drop_duplicates()
df_ = df_[df_.sem_sim.notna()]
df_

df_=df_.rename(columns={'question_1': 'sentence1', 'question_2': 'sentence2', 'sem_sim': 'label'})

df_labeled = df_[df_.label==1].append(df_[df_.label==0], sort=False, ignore_index=True)
# df_labeled['idx'] = df_labeled.index
df_labeled.label = df_labeled.apply(lambda x: int(x.label), axis=1)
df_labeled

,sentence1,sentence2,similarity,label
0,"""WTO là tên viết tắt của tổ chức nào sau đây?\n|Tổ chức Thương mại thế giới.|Liên minh châu Âu.|Hiệp ước tự do thương mại Bắc Mĩ.|Diễn đàn hợp tác kinh tế châu Á – Thái Bình Dương.Đáp án""","""WTO là tên viết tắt của tổ chức|Thương mại thế giới.|Quỹ tiền tệ quốc tế.|Khu vực mậu dịch tự do châu Á.|Diễn đàn hợp tác các kinh tế châu Á -Thái Bình Dương.""",0.602151,1
1,"""Nhằm thúc đẩy sự phát triển kinh tế - xã hội các nước đang phát triển đã tiến hành\n|Nhận chuyển giao các công nghệ lỗi thời, gây ô nhiễm.|Đón đầu được công nghệ hiện đại và áp dụng vào sản xuất.|Mở rộng thị trường buôn bán với nhiều nước trên thế giới.|Phá bỏ hàng rào thuế quan, đẩy mạnh xuất khẩu hàng hóa.Đáp án""","""Trong bối cảnh toàn cầu hóa, các nước đang phát triển đã nhanh chóng nắm bắt được cơ hội nào dưới đây nhằm thúc đẩy sự phát triển kinh tế - xã hội?\n|Nhận chuyển giao các công nghệ lỗi thời, gây ô nhiễm.|Đón đầu được công nghệ hiện đại và áp dụng vào sản xuất.|Mở rộng thị trường buôn bán với nhiều nước trên thế giới.|Phá bỏ hàng rào thuế quan, đẩy mạnh xuất khẩu hàng hóa.Đáp án""",0.745407,1
2,"""Đặc điểm nào không đúng với đặc điểm dân cư Hoa Kì?\n|Dân nhập cư chủ yếu có nguồn gốc Châu Phi.|Dân số Hoa Kì đông.|Dân số Hoa Kì tăng nhanh.|Dân số tăng chủ yếu do nhập cư.Đáp án""","""Đặc điểm nào sau đây không đúng với đặc điểm dân cư Hoa Kì?\n|Dân số Hoa Kì tăng nhanh.|Dân nhập cư chủ yếu có nguồn gốc Châu Phi.|Dân số Hoa Kì đông.|Dân số tăng chủ yếu do nhập cư.Đáp án""",0.682540,1
3,"""Yếu tố tự nhiên nào vừa là khó khăn, nhưng đồng thời cũng là thuận lợi cho sự phát triển kinh tế của Hoa Kì?\n|Lãnh thổ rộng lớn.|Có nhiều động đất và núi lửa.|Có tài nguyên thiên nhiên đa dạng.|Có nhiều người nhập cư.Đáp án""","""Yếu tố tự nhiên nào vừa là khó khăn nhưng đồng thời cũng là thuận lợi cho sự phát triển kinh tế của Hoa Kì?\n|Có tài nguyên thiên nhiên đa dạng.|Lãnh thổ rộng lớn.|Có nhiều động đất và núi lửa.|Có nhiều người nhập cư.Đáp án""",0.715556,1
4,"""Một trong những thuận lợi của EU khi hình thành thị trường chung châu Âu là\n|Tăng thuế các nước thành viên khi lưu thông hàng hóa, dịch vụ.|Tăng cường tự do lưu thông về người, hàng hóa, dịch vụ và tiền vốn.|Tăng cường vai trò của từng quốc gia khi buôn bán với các nước ngoài khối.|Tăng thuế giá trị gia tăng khi lưu thông hàng hóa, tiền tệ và dịch vụ.Đáp án""","""Một trong những thuận lợi của EU khi hình thành thị trường chung châu Âu là\n|Tăng thuế giá trị gia tăng khi lưu thông hàng hóa, tiền tệ và dịch vụ.|Tăng cường tự do lưu thông về người, hàng hóa, dịch vụ và tiền vốn.|Tăng thuế các nước thành viên khi lưu thông hàng hóa, dịch vụ.|Tăng cường vai trò của từng quốc gia khi buôn bán với các nước ngoài khối.Đáp án""",0.642659,1
...,...,...,...,...
975,"""Nhiệt độ không khí thay đổi theo bờ Đông và bờ Tây các lục địa do\n \n|hai bờ Đông và Tây của các lục địa có độ cao khác nhau.|chế độ gió thổi ở hai bờ Đông và Tây các lục địa khác nhau.|hai bờ Đông và Tây của lục địa có góc chiếu của tia bức xạ mặt trời khác nhau.|ảnh hưởng của các dòng biển nóng và lạnh ở hai bờ Đông và Tây khác nhau.Đáp án""","""Tại sao nhiệt độ không khí thay đổi theo bờ đông và bờ Tây các lục địa?\n|hai bờ Đông và Tây của các lục địa có độ cao khác nhau.|chế độ gió thổi ở hai bờ Đông và Tây các lục địa khác nhau.|hai bờ Đông và Tây của lục địa có góc chiếu của Tia bức xạ mặt trời khác nhau.|ảnh hưởng của các dòng biển nóng và lạnh ở hai bờ Đông và Tây khác nhau.Đáp án""",0.954023,0
976,"""Vì sao khu vực chịu ảnh hưởng của gió mùa thường có lượng mưa lớn?\n|Gió mùa mùa Đông thường đem mưa đến.|Gió mùa mùa hạ thổi từ biển vào thường xuyên đem mưa đến.|Cả gió mùa mùa hạ và gió mùa mùa Đông đều đem mưa lớn đến.|Thường xuyên chịu ảnh hưởng của áp thấp.Đáp án""","""Khu vực chịu ảnh hưởng của gió mùa thường có lượng mưa lớn vì\n|Gió mùa mùa Đông thường đem mưa đến.|Gió mùa mùa hạ thổi từ biển vào thường xuyên đem mưa đến.|Cả gió mùa mùa hạ và gió mùa mùa Đông đều đem mưa lớn đến.|Thường xuyên 

**Merge old data with augmented data**

In [7]:
dataset = df_labeled.append(augmented_data_only, sort=False, ignore_index=True)
dataset = dataset.drop_duplicates()
dataset.reset_index(drop=True, inplace=True)
dataset

,sentence1,sentence2,similarity,label
0,"""WTO là tên viết tắt của tổ chức nào sau đây?\n|Tổ chức Thương mại thế giới.|Liên minh châu Âu.|Hiệp ước tự do thương mại Bắc Mĩ.|Diễn đàn hợp tác kinh tế châu Á – Thái Bình Dương.Đáp án""","""WTO là tên viết tắt của tổ chức|Thương mại thế giới.|Quỹ tiền tệ quốc tế.|Khu vực mậu dịch tự do châu Á.|Diễn đàn hợp tác các kinh tế châu Á -Thái Bình Dương.""",0.602151,1
1,"""Nhằm thúc đẩy sự phát triển kinh tế - xã hội các nước đang phát triển đã tiến hành\n|Nhận chuyển giao các công nghệ lỗi thời, gây ô nhiễm.|Đón đầu được công nghệ hiện đại và áp dụng vào sản xuất.|Mở rộng thị trường buôn bán với nhiều nước trên thế giới.|Phá bỏ hàng rào thuế quan, đẩy mạnh xuất khẩu hàng hóa.Đáp án""","""Trong bối cảnh toàn cầu hóa, các nước đang phát triển đã nhanh chóng nắm bắt được cơ hội nào dưới đây nhằm thúc đẩy sự phát triển kinh tế - xã hội?\n|Nhận chuyển giao các công nghệ lỗi thời, gây ô nhiễm.|Đón đầu được công nghệ hiện đại và áp dụng vào sản xuất.|Mở rộng thị trường buôn bán với nhiều nước trên thế giới.|Phá bỏ hàng rào thuế quan, đẩy mạnh xuất khẩu hàng hóa.Đáp án""",0.745407,1
2,"""Đặc điểm nào không đúng với đặc điểm dân cư Hoa Kì?\n|Dân nhập cư chủ yếu có nguồn gốc Châu Phi.|Dân số Hoa Kì đông.|Dân số Hoa Kì tăng nhanh.|Dân số tăng chủ yếu do nhập cư.Đáp án""","""Đặc điểm nào sau đây không đúng với đặc điểm dân cư Hoa Kì?\n|Dân số Hoa Kì tăng nhanh.|Dân nhập cư chủ yếu có nguồn gốc Châu Phi.|Dân số Hoa Kì đông.|Dân số tăng chủ yếu do nhập cư.Đáp án""",0.682540,1
3,"""Yếu tố tự nhiên nào vừa là khó khăn, nhưng đồng thời cũng là thuận lợi cho sự phát triển kinh tế của Hoa Kì?\n|Lãnh thổ rộng lớn.|Có nhiều động đất và núi lửa.|Có tài nguyên thiên nhiên đa dạng.|Có nhiều người nhập cư.Đáp án""","""Yếu tố tự nhiên nào vừa là khó khăn nhưng đồng thời cũng là thuận lợi cho sự phát triển kinh tế của Hoa Kì?\n|Có tài nguyên thiên nhiên đa dạng.|Lãnh thổ rộng lớn.|Có nhiều động đất và núi lửa.|Có nhiều người nhập cư.Đáp án""",0.715556,1
4,"""Một trong những thuận lợi của EU khi hình thành thị trường chung châu Âu là\n|Tăng thuế các nước thành viên khi lưu thông hàng hóa, dịch vụ.|Tăng cường tự do lưu thông về người, hàng hóa, dịch vụ và tiền vốn.|Tăng cường vai trò của từng quốc gia khi buôn bán với các nước ngoài khối.|Tăng thuế giá trị gia tăng khi lưu thông hàng hóa, tiền tệ và dịch vụ.Đáp án""","""Một trong những thuận lợi của EU khi hình thành thị trường chung châu Âu là\n|Tăng thuế giá trị gia tăng khi lưu thông hàng hóa, tiền tệ và dịch vụ.|Tăng cường tự do lưu thông về người, hàng hóa, dịch vụ và tiền vốn.|Tăng thuế các nước thành viên khi lưu thông hàng hóa, dịch vụ.|Tăng cường vai trò của từng quốc gia khi buôn bán với các nước ngoài khối.Đáp án""",0.642659,1
...,...,...,...,...
1485,"Những ý kiến nào sau đây là đúng với hoạt động thương mại nước ngoài của LB Nga?\n|.Giá trị xuất khẩu luôn luôn cân bằng với giá trị nhập khẩu.|.Hàng hóa xuất khẩu chính là hải sản, hàng hóa công nghiệp nhẹ.|.Hàng hóa nhập khẩu là dầu khí.|.Tổng kim ngạch ngoại thương tiếp tục tăng. Trả lời","Phát biểu nào sau đây là đúng đối với thương mại nước ngoài của Nga LB? | Giá trị xuất khẩu luôn được cân bằng với giá trị nhập khẩu. | Các sản phẩm xuất khẩu chính là hải sản, hàng hóa công nghiệp nhẹ. Hàng hóa nhập khẩu là mỏ dầu khí. | Tổng doanh thu từ ngoại thương. Tăng.",0.921619,1
1486,"Giới hạn quan trọng nhất về điều kiện tự nhiên của Nhật Bản và tài nguyên thiên nhiên là\n|.Bờ biển dài, nhiều vũng nước.|.Khí hậu giao rõ từ Bắc đến Nam.|.Khoáng chất kém.|.Nhiều hòn đảo lớn và nhỏ nhưng xa nhau.","Trò đùa lớn nhất về điều kiện tự nhiên của Nhật Bản và tài nguyên thiên nhiên là một bờ biển dài, nhiều vũng nước vũ trụ. | Khí hậu được giao rõ ràng từ Bắc vào Nam. | Khoáng chất kém. | Nhiều hòn đảo lớn, nhỏ nhưng xa, nhưng ở xa.",0.966490,1
1487,"Mùa đông kéo dài, lạnh và có rất nhiều tuyết là đặc điểm khí hậu của vùng khí hậu Nhật Bản sau đây?\n|.Bắc Nhật Bản.|.Nam Nhật Bản.|.Khu vực trung tâm Nhật Bản.|.Tiếng Nhật 

**Train test split**

In [8]:
!pip install sklearn
from sklearn.model_selection import train_test_split

In [9]:
df_train, df_test = train_test_split(dataset, test_size=0.2, random_state = 8, shuffle=True)
df_train, df_val = train_test_split(dataset, test_size=0.25, random_state = 8)

In [10]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(1117, 4)
(373, 4)
(298, 4)


In [11]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [12]:
df_train

,sentence1,sentence2,similarity,label
0,"Bản chất của vùng núi phía đông bắc khác với phía tây bắc tại điểm | Mùa đông ít lạnh hơn nhưng khô ráo. | Chủ nhật đến sớm, đôi khi có một cơn gió tây. | Mùa đông lạnh đến sớm và kết thúc muộn. | Người giữ lạnh do chiều caocủa địa hình.","Bản chất của khu vực phía đông bắc khác với phía tây bắc từ quan điểm tây bắc | Mùa đông ít lạnh hơn nhưng khô hơn | Chủ nhật đến sớm, đôi khi gió gió, giảm lượng mưa | mùa đông lạnh hơn so với vùng núi thấp | lạnh Khí hậu chủ yếu là do chiều cao của mặt đất",0.945276,1
1,"""Ecđixơn gây|ức chế Sự lột xác của sâu bướm, kích thích sâu biến thành nhộng và bướm|ức chế Sự lột xác của sâu bướm, kìm hãm sâu biến thành nhộng và bướm|lột xác của sâu bướm, kích thích sâu biến thành nhộng và bướm|lột xác của sâu bướm, ức chế sâu biến thành nhộng và bướm""","""Juvenin gây|lột xác của sâu bướm, kích thích sâu biến thành nhộng và bướm|ức chế sâu biến thành nhộng và bướm|ức chế sự lột xác của sâu bướm, kích thích sâu biến thành nhộng và bướm|ức chế sự lột xác của sâu bướm, kìm hãm sđu biến thành nhộng và bướm""",0.722628,0
2,"""Với chiến thắng trong phong trào của quân dân miền Nam đã làm phá sản chiến lược chiến tranh nào của Mĩ ? |""Chiến tranh đơn phương"".|""Chiến tranh đặc biệt"".|""Chiến tranh cục bộ"".|""Việt Nam hoá"" chiến tranh.""","""Chiến thắng Bình Giã (12 - 1964) đã góp phần làm phá sản chiến lược chiến tranh nào của Mĩ?\t |""Chiến tranh đơn phương""|""Chiến tranh đặc biệt""|""Chiến tranh cục bộ""|""Việt Nam hóa"" chiến tranh""",0.774038,0
3,"""Hình thức tổ chức lãnh thổ công nghiệp ở nước ta mới được hình thành từ những năm 90 của thế kỉ XX là|Điểm công nghiệp|Vùng công nghiệp|Khu công nghiệp|Trung tâm công nghiệp""","""Hình thức tổ chức lãnh thổ công nghiệp ở trình độ cao nhất, có không gian rộng lớn là|Điểm công nghiệp.|Khu công nghiệp.|Trung tâm công nghiệp.|Vùng công nghiệp.""",0.628571,0
4,"""Lực lượng nào tham gia đông nhất trong khởi nghĩa nông dân Yên Thế?|Công nhân.|Nông dân.|Các dân tộc sống ở miền núi.|Nông dân và công nhân.""","""Lực lượng nào tham gia đông nhất trong khởi nghĩa nông dân Yên Thế?|Công nhân.|Nông dân.|Các dân tộc sống ở miền núi.|Nông dân và công nhân.""",1.000000,1
...,...,...,...,...
1112,Một chiếc xe của I-Li-A đã bán cho các nước EU khác không phải nộp thuế\n|.Lưu thông hàng hóa miễn phí.|.Tự do di chuyển.|.Tự do lưu thông dịch vụ.|.Tự do lưu thông vốn.,"Một công ty vận tải ở Tây Ban Nha có thể thực hiện hợp đồng tại Pháp với tư cách là một công ty ở Pháp, đó là biểu hiện của tự do dưới đây?\n|.Di chuyển hàng hóa miễn phí.|.Sự tự do di chuyển.|.Tự do lưu thông dịch vụ.|.Tự do lưu thông vốn.",0.825642,1
1113,"""""Lần đầu tiên trong lịch sử, một nước thuộc địa nhỏ yếu đã đánh thắng một nước thực dân hùng mạnh...."" . Đó là câu nói của ai?|Võ Nguyên Giáp.|Chủ tịch Hồ Chí Minh.|Trường Chinh.|Phạm Văn Đồng.""","""“Lần đầu tiên trong lịch sử, một nước thuộc địa nhỏ yếu đã đánh thắng một nước thực dân hùng mạnh . ..”. Hãy cho biết đây là câu nói của ai?|Võ Nguyên Giáp.|Trường Chinh.|Chủ Tịch Hồ Chí Minh.|Phạm Văn Đồng.""",0.779904,1
1114,"""Từ Ô-đét-xa đến Mum-bai, đi vòng qua châu phi mất 11818 hải lí,nếu đi qua kênh Xuy-ê chỉ mất 4198 hải lí, vậy quãng đường được rút ngắn khoảng (%)|58%.|70%.|42%.|50%.""","""Từ Niu-Iooc đến Xít-ni, đi vòng qua Nam Mĩ là 13051 hải lí, nếu đi qua kênh Panama chỉ mất 9692 hải lí, vậy quãng đường được rút ngắn khoảng ( % )|60%.|70%.|26%.|50%.""",0.744048,0
1115,"Từ năm 1946 đến 1980, đó là ba lần Quốc hội thông qua hiến pháp, Hiến pháp nào? Hiến pháp 1946; Hiến pháp 1959; Hiến pháp 1980. | Hiến pháp 1946; Hiến pháp 1960; Hiến pháp 1980; Hiến pháp 1980; Hiến pháp 1980; Hiến pháp 1980; |Hiến pháp 1946; Hiến pháp năm 1975; Hiến pháp 1980. | Hiến pháp 1946; Hiến pháp 1976; Hiến pháp 1980.","Từ năm 1946 đến 1980, ba lần đại hội thông qua hiến pháp, mà hiến pháp làm gì? | 1946 Hiến pháp, Hiến pháp năm 1959; Hiến pháp năm 1980. | 1946 Hiến pháp năm 1960; Hiến pháp năm 1975. Hiến pháp năm 1945. Hiến phá

# Load Bert Model

In [13]:
from transformers import BertTokenizer, BertModel

**Tokenize for our dataset using BertTokenizer**

In [14]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='bert-base-cased'):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(bert_model)  

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = str(self.data.loc[index, 'sentence1'])
        sent2 = str(self.data.loc[index, 'sentence2'])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'label']
            return token_ids, attn_masks, token_type_ids, label  
        else:
            return token_ids, attn_masks, token_type_ids

**SentencePairClassifier**
Using a nn.Linear for classification layer

In [15]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="bert-base-cased", freeze_bert=False):
        super(SentencePairClassifier, self).__init__()
        #  Instantiating BERT-based model object
        self.bert_layer = BertModel.from_pretrained(bert_model, return_dict=False)
        hidden_size = 768

        # Freeze bert layers and only train the classification layer weights
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False

        # Classification layer
        self.cls_layer = nn.Linear(hidden_size, 1)

        self.dropout = nn.Dropout(p=0.1)

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''

        # Feeding the inputs to the BERT-based model to obtain contextualized representations
        # cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)
        cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)

        # Feeding to the classifier layer the last layer hidden-state of the [CLS] token further processed by a
        # Linear Layer and a Tanh activation. The Linear layer weights were trained from the sentence order prediction (ALBERT) or next sentence prediction (BERT)
        # objective during pre-training.
        logits = self.cls_layer(self.dropout(pooler_output))
        
        return logits

In [16]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    net.eval()

    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            count += 1

    return mean_loss / count

In [17]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...


In [18]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []

    scaler = GradScaler()

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients
            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                scaler.step(opti)
                # Updates the scale for next iteration.
                scaler.update()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                opti.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

    # Saving the model
    path_to_model='models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(bert_model, lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()

**Parameters**

In [19]:
bert_model = "bert-base-cased"
freeze_bert = False  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = 256
bs = 8
iters_to_accumulate = 2  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = 2e-5
epochs = 4

# Training and Validation

In [20]:
#  Set all seeds to make reproducible results
set_seed(1)

# Creating instances of training and validation set
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=bs, num_workers=5,shuffle=True)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=5,shuffle=True)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

Reading training data...


Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Reading validation data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

...


Iteration 112/140 of epoch 4 complete. Loss : 0.14794190307813032 


100%|██████████| 140/140 [03:15<00:00,  1.40s/it]



Iteration 140/140 of epoch 4 complete. Loss : 0.1271812704923962 


100%|██████████| 47/47 [00:22<00:00,  2.10it/s]



Epoch 4 complete! Validation Loss : 0.3512336974765392
Best validation loss improved from 0.3687236917779801 to 0.3512336974765392

The model has been saved in models/bert-base-cased_lr_2e-05_val_loss_0.35123_ep_4.pt


# Predictions

In [21]:
def get_probs_from_logits(logits):
    """
    Converts a tensor of logits into an array of probabilities by applying the sigmoid function
    """
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

def test_prediction(net, device, dataloader, with_labels=True, result_file="results/output.txt"):
    """
    Predict the probabilities on a dataset with or without labels and print the result in a file
    """
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    with torch.no_grad():
        if with_labels:
            for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
        else:
            for seq, attn_masks, token_type_ids in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()

    w.writelines(str(prob)+'\n' for prob in probs_all)
    w.close()

In [23]:
path_to_model = 'models/bert-base-cased_lr_2e-05_val_loss_0.35123_ep_4.pt'  

path_to_output_file = 'results/output.txt'

print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=5)

model = SentencePairClassifier(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print()
print("Loading the weights of the model...")
model.load_state_dict(torch.load(path_to_model))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=True,  # set the with_labels parameter to False if your want to get predictions on a dataset without labels
                result_file=path_to_output_file)
print()
print("Predictions are available in : {}".format(path_to_output_file))

Reading test data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Loading the weights of the model...
Predicting on test data...


100%|██████████| 38/38 [00:18<00:00,  2.08it/s]


Predictions are available in : results/output.txt


# Evaluation

In [24]:
path_to_output_file = 'results/output.txt'  # path to the file with prediction probabilities

labels_test = df_test['label']  # true labels

probs_test = pd.read_csv(path_to_output_file, header=None)[0]  # prediction probabilities
threshold = 0.5   # you can adjust this threshold for your own dataset
preds_test=(probs_test>=threshold).astype('uint8') # predicted labels using the above fixed threshold

metric = load_metric("glue", "mrpc")

https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/tmpvswvjvp6


Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

storing https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py in cache at /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
creating metadata file for /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
Checking /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py for additional imports.
Creating main folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/metrics/glue
Creating specific version folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/mod

In [25]:
preds_test

0      0
1      0
2      1
3      0
4      0
      ..
293    0
294    0
295    0
296    1
297    0
Name: 0, Length: 298, dtype: uint8

In [26]:
# Compute the accuracy and F1 scores
metric._compute(predictions=preds_test, references=labels_test)

{'accuracy': 0.8187919463087249, 'f1': 0.8014705882352942}